This notebook takes Kristen's post-processed CESM output (monthly time series) and formats it to be read by FEISTY

Note that both the baselines and the initial conditions contain several grid cells where 0 < biomass < 1e-300, and such small values result in underflow (I suppose 1e-308 is a better cut-off, but 1e-300 is still really small).
For the baselines, I faithfully reproduce the (tiny) values matlab outputs, but treat them as 0 when doing comparisons in `matlab-comparison.ipynb`.
For initial conditions, I drop anything smaller than 1e-300 to 0 (this threshold is a variable named `thres` in `[2]`).

In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import xarray as xr

In [2]:
# Common parameter of minimum allowed non-zero biomass
thres = 1e-300

# Helper function for common task of generating netCDF file
def write_netcdf(ds, filename):
    if os.path.isfile(filename):
        print(f'Removing {filename} before writing new copy')
        os.remove(filename)
    ds.to_netcdf(filename)

### Generate forcing (does not contain biomass) from CESM output

In [3]:
# /glade/work/kristenk/fish-offline/g.e11_LENS.GECOIAF.T62_g16.009.FIESTY-forcing.nc
cesm_output = os.path.join(
    os.sep,
    "glade",
    "work",
    "kristenk",
    "fish-offline",
    "g.e11_LENS.GECOIAF.T62_g16.009.FIESTY-forcing.nc",
)
ds_in = xr.open_dataset(cesm_output, decode_times=False)

vert_grid_file = os.path.join(
    os.sep,
    'glade',
    'p',
    'cesmdata',
    'cseg',
    'releases',
    'cesm2_0',
    'components',
    'pop',
    'input_templates',
    'gx1v7_vert_grid',
)

spinup_forcing_file_name = os.path.join(
    '..', 'input_files', 'feisty_input_from_FOSI_monthly_1year.nc'
)

forcing_file_name = os.path.join('..', 'input_files', 'feisty_input_from_FOSI_monthly.nc')

depth = []
with open(vert_grid_file) as f:
    for line in f:
        depth.append(float(line.strip().split()[-1]))

print(f'column depths are {depth}')
ds_in

column depths are [10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 110.0, 120.0, 130.0, 140.0, 150.0, 160.0, 170.1968, 180.7613, 191.8212, 203.4993, 215.9234, 229.2331, 243.5845, 259.1558, 276.1526, 294.8147, 315.4237, 338.3122, 363.8747, 392.5804, 424.9888, 461.7665, 503.7068, 551.7491, 606.9966, 670.7286, 744.398, 829.6069, 928.0435, 1041.3682, 1171.0402, 1318.0936, 1482.9008, 1664.9921, 1863.0144, 2074.874, 2298.0391, 2529.9041, 2768.0986, 3010.6709, 3256.1387, 3503.4487, 3751.8921, 4001.0117, 4250.5244, 4500.2603, 4750.1196, 5000.0464, 5250.0088, 5499.9897]


<xarray.Dataset>
Dimensions:             (nlat: 384, nlon: 320, time: 816)
Coordinates:
    TLONG               (nlat, nlon) float64 ...
    TLAT                (nlat, nlon) float64 ...
  * time                (time) float64 9.092e+04 9.094e+04 ... 1.157e+05
Dimensions without coordinates: nlat, nlon
Data variables:
    KMT                 (nlat, nlon) float64 ...
    TAREA               (nlat, nlon) float64 ...
    TEMP_150m           (time, nlat, nlon) float32 ...
    diazC_150m          (time, nlat, nlon) float32 ...
    zooC_150m           (time, nlat, nlon) float32 ...
    zoo_loss_150m       (time, nlat, nlon) float32 ...
    diatC_150m          (time, nlat, nlon) float32 ...
    spC_150m            (time, nlat, nlon) float32 ...
    TEMP_bottom         (time, nlat, nlon) float32 ...
    POC_FLUX_IN_bottom  (time, nlat, nlon) float32 ...
Attributes:
    nsteps_total:    2263750
    tavg_sum:        2678400.0
    title:           g.e11_LENS.GECOIAF.T62_g16.009
    start_time:      This dataset was created on 2015-07-15 at 00:58:12.7
    history:         none
    Conventions:     CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-curr...
    source:          CCSM POP2, the CCSM Ocean Component
    cell_methods:    cell_methods = time: mean ==> the variable values are av...
    calendar:        All years have exactly  365 days.
    tavg_sum_qflux:  2678400.0
    contents:        Diagnostic and Prognostic Variables
    revision:        $Id: tavg.F90 41939 2012-11-14 16:37:23Z mlevy@ucar.edu $

In [4]:
time0 = np.concatenate([np.array([249 * 365]), ds_in["time"].data[:-1]])
time1 = ds_in["time"].data
# ds_in = xr.Dataset(
new_time = 0.5 * (time0 + time1) - 365
new_time[:24]

array([90535.5, 90565. , 90594.5, 90625. , 90655.5, 90686. , 90716.5,
       90747.5, 90778. , 90808.5, 90839. , 90869.5, 90900.5, 90930. ,
       90959.5, 90990. , 91020.5, 91051. , 91081.5, 91112.5, 91143. ,
       91173.5, 91204. , 91234.5])

#### Reshape, drop land, and do unit conversion

From matlab:

```
%% fraction large phyto -> frac large zoo
fracL = diatC_150m ./ (diatC_150m + spC_150m + diazC_150m + eps);

LzooC_150m = fracL .* zooC_150m;
Lzoo_loss_150m = fracL .* zoo_loss_150m;
```

Some units:

```
%% Units
% meso zoo: nmolC cm-2 to g(WW) m-2
zooC_150m = zooC_150m * 1e-9 * 1e4 * 12.01 * 9.0;
LzooC_150m = LzooC_150m * 1e-9 * 1e4 * 12.01 * 9.0;

% meso zoo mortality: nmolC cm-2 s-1 to g(WW) m-2 d-1
zoo_loss_150m = zoo_loss_150m * 1e-9 * 1e4 * 12.01 * 9.0 * 60 * 60 * 24;
Lzoo_loss_150m = Lzoo_loss_150m * 1e-9 * 1e4 * 12.01 * 9.0 * 60 * 60 * 24;
```

Note about `eps`:

```
>> format long
>> disp(eps)
     2.220446049250313e-16
```

(I replaced with an `np.where()`)

In [5]:
%%time


def my_reshape(da):
    if 'time' in da.dims:
        return np.reshape(da.data, (816, 384 * 320))
    else:
        return np.reshape(da.data, 384 * 320)


biomass_conversion = 1e-9 * 1e4 * 12.01 * 9.0  # nmolC cm-2 -> G(WW) m-2
flux_conversion = biomass_conversion * 60 * 60 * 24  # nmolC cm-2 s-1 to g(WW) m-2 d-1

# Grid variables
mask = my_reshape(ds_in["KMT"]) > 0
X = my_reshape(ds_in["TLONG"])[mask]
lat = my_reshape(ds_in["TLAT"])[mask]
kmt = my_reshape(ds_in["KMT"].astype('int'))[mask] - 1
dep = [depth[k] for k in kmt]
dep

# Intermediate forcing variables
# (zooC and zoo_mort are scaled by fraction of autoC coming from diatoms)
spC = my_reshape(ds_in["spC_150m"])[:, mask]
diatC = my_reshape(ds_in["diatC_150m"])[:, mask]
diazC = my_reshape(ds_in["diazC_150m"])[:, mask]
totC = spC + diatC + diazC
fracL = np.where(totC > 0, diatC / totC, 0)

# Forcing variables
T_pelagic = my_reshape(ds_in["TEMP_150m"])[:, mask]
T_bottom = my_reshape(ds_in["TEMP_bottom"])[:, mask]
poc_flux_bottom = my_reshape(ds_in["POC_FLUX_IN_bottom"])[:, mask] * flux_conversion
zooC = fracL * my_reshape(ds_in["zooC_150m"])[:, mask] * biomass_conversion
zoo_mort = fracL * my_reshape(ds_in["zoo_loss_150m"])[:, mask] * flux_conversion

CPU times: user 7.37 s, sys: 7.88 s, total: 15.3 s
Wall time: 30.4 s


In [6]:
metadata = dict()

metadata['time'] = dict()
metadata['time']['long_name'] = 'time'
metadata['time']['units'] = 'days since 0001-01-01 00:00:00'
metadata['time']['calendar'] = 'noleap'
metadata['time']['axis'] = 'T'

metadata['lat'] = dict()
metadata['lat']['long_name'] = 'latitude'
metadata['lat']['standard_name'] = 'latitude'
metadata['lat']['units'] = 'degrees_north'
metadata['lat']['axis'] = 'Y'

metadata['dep'] = dict()
metadata['dep']['long_name'] = '    bottom depth'
metadata['dep']['standard_name'] = '    bottom depth'
metadata['dep']['units'] = 'm'
metadata['dep']['axis'] = 'Z'

metadata['T_pelagic'] = dict()
metadata['T_pelagic']['long_name'] = 'Pelagic mean temperature'
metadata['T_pelagic']['units'] = 'degrees C'

metadata['T_bottom'] = dict()
metadata['T_bottom']['long_name'] = 'Bottom temperature'
metadata['T_bottom']['units'] = 'degrees C'

metadata['poc_flux_bottom'] = dict()
metadata['poc_flux_bottom']['long_name'] = 'Particulate organic matter flux to seafloor'
metadata['poc_flux_bottom']['units'] = 'g m-2 d-1'

metadata['zooC'] = dict()
metadata['zooC']['long_name'] = 'Biomass of mesozooplankton'
metadata['zooC']['units'] = 'g m-2'

metadata['zoo_mort'] = dict()
metadata['zoo_mort']['long_name'] = 'Mortality loss of mesozooplankton'
metadata['zoo_mort']['units'] = 'g m-2 d-1'

In [7]:
ds_forcing = xr.Dataset(
    {
        'lat': (['X'], lat),
        'dep': (['X'], dep),
        'T_pelagic': (['time', 'X'], T_pelagic),
        'T_bottom': (['time', 'X'], T_bottom),
        'poc_flux_bottom': (['time', 'X'], poc_flux_bottom),
        'zooC': (['time', 'X'], zooC),
        'zoo_mort': (['time', 'X'], zoo_mort),
    }
).assign_coords(time=new_time, X=X)
ds_forcing["time"].attrs = ds_in["time"].attrs
ds_forcing["time"].attrs["axis"] = "T"
for var in metadata:
    ds_forcing[var].attrs = metadata[var]
    ds_forcing[var].encoding['_FillValue'] = None
ds_forcing

<xarray.Dataset>
Dimensions:          (X: 85813, time: 816)
Coordinates:
  * time             (time) float64 9.054e+04 9.056e+04 ... 1.153e+05 1.153e+05
  * X                (X) float64 320.6 321.7 322.8 194.6 ... 302.5 302.7 302.9
Data variables:
    lat              (X) float64 -78.15 -78.15 -78.15 ... 74.87 74.78 74.68
    dep              (X) float64 829.6 829.6 829.6 503.7 ... 30.0 30.0 30.0 30.0
    T_pelagic        (time, X) float32 -1.676 -1.676 -1.675 ... -1.846 -1.85
    T_bottom         (time, X) float32 -1.737 -1.733 -1.729 ... -1.849 -1.854
    poc_flux_bottom  (time, X) float32 0.2883 0.2961 ... 0.001433 0.001433
    zooC             (time, X) float32 9.702 9.738 9.807 ... 0.4748 0.4748
    zoo_mort         (time, X) float32 0.7604 0.7679 0.7748 ... 0.00406 0.004059

#### Also generate forcing for spinup

Just use first 12 months, and call it year 0001 instead of 0249

In [8]:
ds_forcing_spinup = ds_forcing.isel(time=slice(0, 12)).assign_coords(
    time=ds_forcing.time.data[:12] - 248 * 365
)
ds_forcing_spinup.time.attrs = ds_forcing.time.attrs

ds_forcing_spinup

<xarray.Dataset>
Dimensions:          (X: 85813, time: 12)
Coordinates:
  * time             (time) float64 15.5 45.0 74.5 105.0 ... 288.5 319.0 349.5
  * X                (X) float64 320.6 321.7 322.8 194.6 ... 302.5 302.7 302.9
Data variables:
    lat              (X) float64 -78.15 -78.15 -78.15 ... 74.87 74.78 74.68
    dep              (X) float64 829.6 829.6 829.6 503.7 ... 30.0 30.0 30.0 30.0
    T_pelagic        (time, X) float32 -1.676 -1.676 -1.675 ... -1.817 -1.816
    T_bottom         (time, X) float32 -1.737 -1.733 -1.729 ... -1.818 -1.817
    poc_flux_bottom  (time, X) float32 0.2883 0.2961 0.2962 ... 0.00137 0.001371
    zooC             (time, X) float32 9.702 9.738 9.807 ... 0.4683 0.4683
    zoo_mort         (time, X) float32 0.7604 0.7679 ... 0.003963 0.003963

In [9]:
%%time

write_netcdf(ds_forcing_spinup, spinup_forcing_file_name)
ds_read_forcing = xr.open_dataset(spinup_forcing_file_name)
ds_read_forcing

Removing ../input_files/feisty_input_from_FOSI_monthly_1year.nc before writing new copy
CPU times: user 301 ms, sys: 1.4 s, total: 1.7 s
Wall time: 8.88 s


<xarray.Dataset>
Dimensions:          (X: 85813, time: 12)
Coordinates:
  * time             (time) object 0001-01-16 12:00:00 ... 0001-12-16 12:00:00
  * X                (X) float64 320.6 321.7 322.8 194.6 ... 302.5 302.7 302.9
Data variables:
    lat              (X) float64 ...
    dep              (X) float64 ...
    T_pelagic        (time, X) float32 ...
    T_bottom         (time, X) float32 ...
    poc_flux_bottom  (time, X) float32 ...
    zooC             (time, X) float32 ...
    zoo_mort         (time, X) float32 ...

In [ ]:
%%time

write_netcdf(ds_forcing, forcing_file_name)
ds_read_forcing = xr.open_dataset(forcing_file_name)
ds_read_forcing

### Get initial conditions out of `.mat` file

Probably should generate initial conditions directly from CESM output as well, but for now using IC generated from matlab output